# 5. 分支鏈 (Chains Branching) - Ollama 版本

本範例展示根據條件選擇不同處理路徑，建立一個智能學習助手，根據學生問題類型提供不同的學習建議。

## 學習重點
- 使用 RunnableBranch 實現條件分支
- 學習如何根據輸入內容選擇不同的處理路徑
- 理解分支邏輯的設計模式
- 掌握複雜條件判斷的實現方法


In [ ]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain_ollama.llms import OllamaLLM

# 載入環境變數
load_dotenv()

# 建立 Ollama 模型
model = OllamaLLM(model="llama3.2:latest")


In [ ]:
# 定義不同學習問題類型的提示模板

# 概念解釋模板
concept_explanation_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，擅長用簡單易懂的方式解釋複雜概念。"),
        ("human", "請解釋這個概念：{question}"),
    ]
)

# 解題指導模板
problem_solving_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，擅長指導學生解題步驟。"),
        ("human", "請指導我如何解決這個問題：{question}"),
    ]
)

# 學習建議模板
study_advice_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，擅長提供學習方法和建議。"),
        ("human", "請給我學習建議：{question}"),
    ]
)

# 複雜問題模板
complex_question_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，這個問題比較複雜，需要詳細分析。"),
        ("human", "這是一個複雜問題，請詳細分析：{question}"),
    ]
)

print("學習助手模板已定義：")
print("1. concept_explanation_template: 概念解釋")
print("2. problem_solving_template: 解題指導")
print("3. study_advice_template: 學習建議")
print("4. complex_question_template: 複雜問題")


In [ ]:
# 定義問題分類模板
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，請將學生問題分類為：概念解釋、解題指導、學習建議或複雜問題。"),
        ("human", "請將這個學習問題分類：{question}"),
    ]
)

print("問題分類模板已定義：classification_template")


In [ ]:
# 定義分支處理邏輯
# 使用 RunnableBranch 根據分類結果選擇不同的處理路徑
branches = RunnableBranch(
    # 概念解釋分支
    (
        lambda x: "概念" in x or "解釋" in x or "什麼是" in x,
        concept_explanation_template | model | StrOutputParser()
    ),
    # 解題指導分支
    (
        lambda x: "解題" in x or "如何" in x or "怎麼" in x or "步驟" in x,
        problem_solving_template | model | StrOutputParser()
    ),
    # 學習建議分支
    (
        lambda x: "學習" in x or "方法" in x or "建議" in x or "技巧" in x,
        study_advice_template | model | StrOutputParser()
    ),
    # 預設分支（複雜問題）
    complex_question_template | model | StrOutputParser()
)

print("分支處理邏輯已定義：")
print("1. 概念問題 → 概念解釋")
print("2. 解題問題 → 解題指導")
print("3. 學習問題 → 學習建議")
print("4. 其他情況 → 複雜問題分析")


In [ ]:
# 建立分類鏈
classification_chain = classification_template | model | StrOutputParser()

# 組合分類和回應生成鏈
chain = classification_chain | branches

print("完整的學習助手鏈已建立！")
print("鏈的流程：學習問題 → 分類 → 選擇處理路徑 → 生成學習建議")


In [ ]:
# 測試不同類型的學習問題
test_questions = [
    "什麼是機器學習？",  # 概念解釋
    "如何解二元一次方程式？",  # 解題指導
    "有什麼好的學習方法？",  # 學習建議
    "請分析這個複雜的數學問題..."  # 複雜問題
]

print("=" * 60)
print("智能學習助手測試結果：")
print("=" * 60)

for i, question in enumerate(test_questions, 1):
    print(f"\n測試 {i}: {question}")
    print("-" * 40)
    result = chain.invoke({"question": question})
    print(f"學習建議: {result}")
    print("=" * 60)


## 💡 重點說明

### 分支鏈的設計模式

1. **分類階段**: 首先對輸入進行分類
2. **條件判斷**: 根據分類結果選擇處理路徑
3. **分支處理**: 每個分支都有專門的處理邏輯
4. **預設處理**: 提供預設分支處理未分類的情況

### RunnableBranch 的使用

```python
branches = RunnableBranch(
    (條件函數1, 處理鏈1),
    (條件函數2, 處理鏈2),
    (條件函數3, 處理鏈3),
    預設處理鏈  # 最後一個是預設分支
)
```

### 條件函數的設計

```python
# 條件函數應該返回 True 或 False
lambda x: "概念" in x or "解釋" in x or "什麼是" in x
```

### 實際應用場景

- **智能學習助手**: 根據問題類型提供不同指導
- **客服系統**: 自動分類客戶問題
- **內容推薦**: 根據用戶偏好推薦內容
- **智能路由**: 根據請求類型選擇處理方式
